In [1]:
# Cellule 1 – Imports de base 

import numpy as np

from env.workshop_env import WorkshopEnv


In [2]:
# Cellule 2 – Créer l’env et afficher un état

env = WorkshopEnv()

obs, info = env.reset()
print("Shape observation :", obs.shape)
print("Première observation :", obs)
print("Action space :", env.action_space)
print("Observation space :", env.observation_space)


Shape observation : (13,)
Première observation : [ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
Action space : Discrete(201)
Observation space : Box(0.0, [1.008e+04 1.000e+00 1.000e+02 1.000e+00 1.000e+02 5.000e+01 5.000e+01
 5.000e+01 5.000e+01 1.008e+04 1.000e+03 1.000e+03 1.000e+03], (13,), float32)


In [3]:
# Cellule 3 – Épisode avec actions aléatoires
# Objectif : vérifier que l’atelier ne plante pas sur 7 jours et voir un total de reward.
# On doit avoir environ 10080 steps si tout est cohérent.

env = WorkshopEnv()
obs, info = env.reset()

done = False
total_reward = 0.0
step = 0

while not done:
    action = env.action_space.sample()  # politique purement aléatoire
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    step += 1

    done = terminated or truncated

print("Nombre de steps :", step)
print("Reward total sur la semaine (policy aléatoire) :", total_reward)


Nombre de steps : 10080
Reward total sur la semaine (policy aléatoire) : -49009.95999999999


In [4]:
# Cellule 4 – Imports SB3 et check_env

# Etape 3 – Brancher Stable-Baselines3 (DQN)
# On va utiliser SB3 pour éviter de recoder tout le DQN 
#(réseau Q, replay buffer, target network, etc.) à la main.

from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv


In [5]:
# Cellule 5 – Vérifier l’environnement avec check_env

# si pas d'affichage, l'environnement respecte bien l’API Gymnasium
env = WorkshopEnv()
check_env(env, warn=True)


In [6]:
# Cellule 6 : vectorisation de l'environnement
# avec DummyVecEnv

from env.workshop_env import WorkshopEnv
from stable_baselines3.common.vec_env import DummyVecEnv

def make_env():
    # Chaque appel crée un nouvel atelier propre (reset complet)
    return WorkshopEnv()

# Environnement vectorisé avec 1 seul atelier
vec_env = DummyVecEnv([make_env])

vec_env


In [7]:
# Cellule 7 : test

obs = vec_env.reset()
print("Observation depuis vec_env :", obs)
print("Shape :", obs.shape)

# une action aléatoire pour tester
action = [vec_env.action_space.sample()]  # liste de taille 1
obs, reward, done, info = vec_env.step(action)
print("Après un step :")
print("obs :", obs)
print("reward :", reward)
print("done :", done)


Observation depuis vec_env : [[ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]]
Shape : (1, 13)
Après un step :
obs : [[ 1.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]]
reward : [-1.]
done : [False]


In [8]:
# Cellule 8 : construction du DQN

from stable_baselines3 import DQN

model = DQN(
    "MlpPolicy",          # Réseau fully-connected (adapté à un vecteur de 13 features)
    vec_env,              # Ton environnement vectorisé
    learning_rate=1e-4,   # Pas d’apprentissage : petit pour éviter l’instabilité
    buffer_size=100_000,  # Taille de la mémoire d’expérience
    learning_starts=5_000,# Avant 5000 interactions, il explore uniquement
    batch_size=256,       # Taille des minibatches
    gamma=0.995,          # Discount factor : horizon long → gamma élevé
    train_freq=4,         # Mise à jour du réseau toutes les 4 steps
    target_update_interval=5_000, # Mise à jour du target-network
    verbose=1             # Afficher les logs
)

model


Using cpu device


In [9]:
# Cellule 9 : entrainement

total_timesteps = 300_000 

model.learn(
    total_timesteps=total_timesteps,
    progress_bar=True  # si ta version de SB3 le supporte
)

model.save("dqn_workshop_v1")


C:\Users\fabri\anaconda3\envs\qlearning\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 503      |
|    time_elapsed     | 80       |
|    total_timesteps  | 40320    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.68     |
|    n_updates        | 8829     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 451      |
|    time_elapsed     | 178      |
|    total_timesteps  | 80640    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.55     |
|    n_updates        | 18909    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

In [10]:
# Cellule 10 : test sur une semaine

env_test = WorkshopEnv()
obs, info = env_test.reset()

done = False
total_reward = 0
step = 0

while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env_test.step(int(action))
    total_reward += reward
    step += 1
    done = terminated or truncated

print("Steps exécutés :", step)
print("Reward total du DQN sur une semaine :", total_reward)


Steps exécutés : 10080
Reward total du DQN sur une semaine : -13963.279999999995


In [11]:
env_debug = WorkshopEnv()
obs, info = env_debug.reset()

total_reward = 0

for t in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env_debug.step(int(action))

    total_reward += reward

    print(
        f"t={t:03d}, action={int(action)}, "
        f"reward={reward:.2f}, total={total_reward:.2f}, "
        f"time={env_debug.time}, raw={env_debug.stock.raw}, "
        f"P1={env_debug.stock.p1}, P2_inter={env_debug.stock.p2_inter}, "
        f"P2={env_debug.stock.p2}, backlog1={env_debug.demande_p1}, "
        f"backlog2={env_debug.demande_p2}, "
        f"busyM1={env_debug.m1.busy}, busyM2={env_debug.m2.busy}"
    )

    if terminated or truncated:
        break


t=000, action=150, reward=-1.00, total=-1.00, time=1, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=001, action=123, reward=-1.00, total=-2.00, time=2, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=002, action=123, reward=-1.00, total=-3.00, time=3, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=003, action=123, reward=-1.00, total=-4.00, time=4, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=004, action=123, reward=-1.00, total=-5.00, time=5, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=005, action=123, reward=-1.00, total=-6.00, time=6, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=006, action=123, reward=-1.00, total=-7.00, time=7, raw=10, P1=0, P2_inter=0, P2=0, backlog1=0, backlog2=0, busyM1=False, busyM2=False
t=007, action=123, reward=-1.00, total=-8